# TensorBoard HyperParameters Logging

Source: https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams

In this notebook, we'll use TensorBoard to compare multiple training runs with different hyper-parameters settings. Let us start by adding the TB extension to jupyter and clearing the logs directory.

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Clear any logs from previous runs
!rm -rf ./tb_log/ 

Now let's import the required modules. We'll use the `hparams` plugin of Keras to explore hyper-parameters settings. This plugin has several other functionalities not touched in this example.

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp

2021-12-22 12:24:21.419584: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-22 12:24:21.419611: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


##### Load the dataset

We'll use again Fashion MNIST for this example.

In [3]:
fashion_mnist = keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


##### Build the model

We'll use a function to build a different version of the model based on some hyperparameters. In particular, we'll use only two hyperparameters here for simplicity:
- The number of units in the first fully-connected layer of our NN (`hparams['num_units']`)
- The dropout rate of that same layer (`hparams['dropout']`)

In [4]:
def try_hp_setting(hparams):
    model = keras.models.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(hparams['num_units'], activation=tf.nn.relu),
        keras.layers.Dropout(hparams['dropout']),
        keras.layers.Dense(10, activation=tf.nn.softmax),
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

##### Train the model

We'll train the model multiple times for different settings of the two hyper-parameters. Each run will be identified by an index (`session_num`). The corresponding logs will be saved in a separate sub-folder for TensorBoard.

The Keras `hparams` plugin has a dedicated callback (`hp.KerasCallback`) which logs the values of the hyper-parameters passed as input associated with each run.

In [5]:
session_num = 0

In [6]:
# for num_units in [512, 1024]:
for num_units in [16, 64]:
  for dropout_rate in [0.1, 0.5]:
        hparams = {'num_units': num_units, 'dropout': dropout_rate}
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print(hparams)
    
        log_dir = 'tb_log/hparam_tuning/' + run_name
        tb_callback = tf.keras.callbacks.TensorBoard(log_dir)
        hp_callback = hp.KerasCallback(log_dir, hparams)
    
        model = try_hp_setting(hparams)
        model.fit(x_train, y_train, validation_split=0.1, epochs=10, callbacks=[tb_callback, hp_callback])
        session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1}


2021-12-22 12:24:23.611826: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-22 12:24:23.611854: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-22 12:24:23.611878: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (matteo-Inspiron-7591-2n1): /proc/driver/nvidia/version does not exist
2021-12-22 12:24:23.612087: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-22 12:24:23.844730: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 16

Epoch 1/10
1688/1688 [==============================] - 2s 889us/step - loss: 0.7031 - accuracy: 0.7601 - val_loss: 0.4688 - val_accuracy: 0.8298
Epoch 2/10
1688/1688 [==============================] - 1s 792us/step - loss: 0.5025 - accuracy: 0.8226 - val_loss: 0.4381 - val_accuracy: 0.8443
Epoch 3/10
1688/1688 [==============================] - 1s 812us/step - loss: 0.4645 - accuracy: 0.8361 - val_loss: 0.4061 - val_accuracy: 0.8562
Epoch 4/10
1688/1688 [==============================] - 1s 757us/step - loss: 0.4444 - accuracy: 0.8401 - val_loss: 0.4008 - val_accuracy: 0.8557
Epoch 5/10
1688/1688 [==============================] - 1s 793us/step - loss: 0.4335 - accuracy: 0.8422 - val_loss: 0.4022 - val_accuracy: 0.8533
Epoch 6/10
1688/1688 [==============================] - 1s 805us/step - loss: 0.4275 - accuracy: 0.8455 - val_loss: 0.3964 - val_accuracy: 0.8555
Epoch 7/10
1688/1688 [==============================] - 1s 803us/step - loss: 0.4145 - accuracy: 0.8499 - val_loss: 0.3985 -

2021-12-22 12:24:38.953444: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 169344000 exceeds 10% of free system memory.


Epoch 1/10
1688/1688 [==============================] - 2s 1ms/step - loss: 1.1691 - accuracy: 0.5585 - val_loss: 0.6193 - val_accuracy: 0.8102
Epoch 2/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.9464 - accuracy: 0.6459 - val_loss: 0.5647 - val_accuracy: 0.8188
Epoch 3/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.8952 - accuracy: 0.6631 - val_loss: 0.5338 - val_accuracy: 0.8160
Epoch 4/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.8730 - accuracy: 0.6716 - val_loss: 0.5085 - val_accuracy: 0.8257
Epoch 5/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.8660 - accuracy: 0.6721 - val_loss: 0.5093 - val_accuracy: 0.8273
Epoch 6/10
1688/1688 [==============================] - 2s 986us/step - loss: 0.8617 - accuracy: 0.6759 - val_loss: 0.5098 - val_accuracy: 0.8225
Epoch 7/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.8460 - accuracy: 0.6826 - val_loss: 0.5154 - val_accurac

2021-12-22 12:24:57.269231: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 169344000 exceeds 10% of free system memory.


Epoch 1/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.5690 - accuracy: 0.8029 - val_loss: 0.4168 - val_accuracy: 0.8523
Epoch 2/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.4180 - accuracy: 0.8511 - val_loss: 0.3946 - val_accuracy: 0.8590
Epoch 3/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.3798 - accuracy: 0.8629 - val_loss: 0.3713 - val_accuracy: 0.8640
Epoch 4/10
1688/1688 [==============================] - 2s 946us/step - loss: 0.3570 - accuracy: 0.8699 - val_loss: 0.3529 - val_accuracy: 0.8710
Epoch 5/10
1688/1688 [==============================] - 2s 940us/step - loss: 0.3425 - accuracy: 0.8749 - val_loss: 0.3488 - val_accuracy: 0.8733
Epoch 6/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.3316 - accuracy: 0.8785 - val_loss: 0.3420 - val_accuracy: 0.8793
Epoch 7/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.3176 - accuracy: 0.8836 - val_loss: 0.3294 - val_accur

2021-12-22 12:25:16.600845: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 169344000 exceeds 10% of free system memory.


Epoch 1/10
1688/1688 [==============================] - 3s 1ms/step - loss: 0.7323 - accuracy: 0.7426 - val_loss: 0.4815 - val_accuracy: 0.8173
Epoch 2/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.5519 - accuracy: 0.8053 - val_loss: 0.4185 - val_accuracy: 0.8440
Epoch 3/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.5119 - accuracy: 0.8162 - val_loss: 0.4186 - val_accuracy: 0.8432
Epoch 4/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.4907 - accuracy: 0.8231 - val_loss: 0.3919 - val_accuracy: 0.8570
Epoch 5/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.4779 - accuracy: 0.8270 - val_loss: 0.3772 - val_accuracy: 0.8603
Epoch 6/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.4689 - accuracy: 0.8301 - val_loss: 0.3750 - val_accuracy: 0.8647
Epoch 7/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.4568 - accuracy: 0.8339 - val_loss: 0.3803 - val_accuracy:

##### Start Tensorboard

Let's start tensorboard and see the results of this experiment. We can clearly inspect the scalar logs as seen in the previous notebook. However, the `HPARAMS` tab provides a more useful visualization.

In [7]:
%tensorboard --logdir './tb_log'